### Modele DNN

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [ ]:
df_pretraited = pd.read_csv("dataPreprocessed.csv", low_memory=False)
df_pretraited.head()

,normality,sourceID_freq,accessed_addr_freq,src_dest_same_addr,src_dest_same_type,src_dest_same_loc,value_movement,value_tempin,value_lightOn,value_heatingcontrol,...,value_movement_present,value_lightOn_present,value_heatingcontrol_present,acc_node_event_present,hour_sin,hour_cos,delta_time,events_per_min,is_night,is_work_hours
0,0,0.040505,0.000003,1,True,1,0.0,-0.753161,0.0,0.0,...,0,0,0,0,-0.258819,0.965926,-0.072967,0.209502,0,0
1,0,0.040505,0.046168,1,True,1,0.0,-0.753161,0.0,0.0,...,0,0,0,1,-0.258819,0.965926,0.220811,0.274155,0,0
2,0,0.040505,0.018952,1,True,1,0.0,-0.753161,0.0,0.0,...,0,0,0,1,-0.258819,0.965926,-0.028332,0.274155,0,0
3,0,0.040505,0.046168,1,True,1,0.0,-0.753161,0.0,0.0,...,0,0,0,1,-0.258819,0.965926,-0.054648,0.274155,0,0
4,0,0.040505,0.046168,1,True,1,0.0,-0.753161,0.0,0.0,...,0,0,0,1,-0.258819,0.965926,-0.072456,0.274155,0,0


#### Split des donnees

In [ ]:
# X = tes features, y = ta target (0=normal, 1=anomalie)
X = df_pretraited.copy()
X = X.drop(columns=['normality'])
y = df_pretraited['normality']


# Split train+val et test (80%-20%)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Split train et validation (80%-20% du train+val)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, stratify=y_train_val, random_state=42
)

print("Train:", len(X_train), "Validation:", len(X_val), "Test:", len(X_test))

NameError: name 'df_pretraited' is not defined

In [8]:
print("Train (%):\n", y_train.value_counts(normalize=True) * 100)
print("\nValidation (%):\n", y_val.value_counts(normalize=True) * 100)
print("\nTest (%):\n", y_test.value_counts(normalize=True) * 100)

NameError: name 'y_train' is not defined

In [9]:
X_train_np = X_train.values
X_val_np   = X_val.values
X_test_np  = X_test.values

y_train_np = y_train.values
y_val_np   = y_val.values
y_test_np  = y_test.values

NameError: name 'X_train' is not defined

In [10]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.array([0, 1]),
    y=y_train_np
)

class_weight_dict = {
    0: class_weights[0],
    1: class_weights[1]
}

class_weight_dict


NameError: name 'y_train_np' is not defined

In [11]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_np.shape[1],)),
    Dropout(0.3),

    Dense(64, activation='relu'),
    Dropout(0.3),

    Dense(1, activation='sigmoid')
])

NameError: name 'Sequential' is not defined

In [12]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', 'Precision', 'Recall', 'AUC']
)


NameError: name 'model' is not defined